In [ ]:
!pip install langchain google-cloud-aiplatform google-auth neo4j > /dev/null

In [ ]:
import os

from langchain.chat_models import ChatVertexAI
from langchain.chains import GraphCypherQAChain

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/run/media/solverbot/repoA/gitFolders/generativeaitrial-trialLC.json"

In [6]:
from langchain.chat_models import ChatVertexAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

In [8]:
graph = Neo4jGraph(
    url="bolt://35.171.193.91:7687",
    username="langc",
    password="password"
)

In [9]:
chain = GraphCypherQAChain.from_llm(
    ChatVertexAI(temperature=0),
    graph=graph,
    verbose=True
)

In [11]:
chain.run("How many products did each customer purchase?")



> Entering new  chain...
Generated Cypher:
MATCH (o:Order)-[:CONTAINS]->(p:Product) RETURN o.shipName, count(p)
Full Context:
[{'o.shipName': 'Save-a-lot Markets', 'count(p)': 116}, {'o.shipName': 'Bottom-Dollar Markets', 'count(p)': 35}, {'o.shipName': 'Tortuga Restaurante', 'count(p)': 29}, {'o.shipName': 'Wellington Importadora', 'count(p)': 19}, {'o.shipName': 'Berglunds snabbköp', 'count(p)': 52}, {'o.shipName': 'Chop-suey Chinese', 'count(p)': 22}, {'o.shipName': 'HILARION-Abastos', 'count(p)': 45}, {'o.shipName': 'LINO-Delicateses', 'count(p)': 35}, {'o.shipName': 'Mère Paillarde', 'count(p)': 32}, {'o.shipName': 'Du monde entier', 'count(p)': 9}]

> Finished chain.


'Save-a-lot Markets purchased 116 products, Bottom-Dollar Markets purchased 35 products, Tortuga Restaurante purchased 29 products, Wellington Importadora purchased 19 products, Berglunds snabbköp purchased 52 products, Chop-suey Chinese purchased 22 products, HILARION-Abastos purchased 45 products, LINO-Delicateses purchased 35 products, Mère Paillarde purchased 32 products, and Du monde entier purchased 9 products.'

In [ ]:
chain.run("Create relationships between customers and orders based on customerID?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Customer)-[:PURCHASED]->(o:Order) RETURN c.customerID, o.orderID
Full Context:
[{'c.customerID': 'ALFKI', 'o.orderID': '10702'}, {'c.customerID': 'ALFKI', 'o.orderID': '11011'}, {'c.customerID': 'ALFKI', 'o.orderID': '10643'}, {'c.customerID': 'ALFKI', 'o.orderID': '10952'}, {'c.customerID': 'ALFKI', 'o.orderID': '10835'}, {'c.customerID': 'ALFKI', 'o.orderID': '10692'}, {'c.customerID': 'ANATR', 'o.orderID': '10625'}, {'c.customerID': 'ANATR', 'o.orderID': '10308'}, {'c.customerID': 'ANATR', 'o.orderID': '10759'}, {'c.customerID': 'ANATR', 'o.orderID': '10926'}, {'c.customerID': 'ANTON', 'o.orderID': '10856'}, {'c.customerID': 'ANTON', 'o.orderID': '10682'}, {'c.customerID': 'ANTON', 'o.orderID': '10365'}, {'c.customerID': 'ANTON', 'o.orderID': '10573'}, {'c.customerID': 'ANTON', 'o.orderID': '10677'}, {'c.customerID': 'ANTON', 'o.orderID': '10507'}, {'c.customerID': 'ANTON', 'o.orderID': '10535'}, {'c.customerID'

'The following relationships between customers and orders have been created based on customerID:\n\nALFKI 10702\nALFKI 11011\nALFKI 10643\nALFKI 10952\nALFKI 10835\nALFKI 10692\nANATR 10625\nANATR 10308\nANATR 10759\nANATR 10926\nANTON 10856\nANTON 106'

In [13]:
# https://github.com/hwchase17/langchain/blob/master/langchain/chains/graph_qa/prompts.py
from langchain.prompts.prompt import PromptTemplate


CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Cypher examples:
# How many streamers are from Norway?
MATCH (s:Stream)-[:HAS_LANGUAGE]->(:Language {{name: 'no'}})
RETURN count(s) AS streamers
# How many products did each customer purchase?
MATCH (cust:Customer)-[:PURCHASED]->(:Order)-[o:ORDERS]->(p:Product),
  (p)-[:PART_OF]->(c:Category {{categoryName:"Produce"}})
RETURN DISTINCT cust.contactName as CustomerName, SUM(o.quantity) AS TotalProductsPurchased
# What categories of food does each supplier supply?
MATCH (s:Supplier)-->(:Product)-->(c:Category)
RETURN s.companyName as Company, collect(distinct c.categoryName) as Categories

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [14]:
chain_language_example = GraphCypherQAChain.from_llm(
    ChatVertexAI(temperature=0), graph=graph, verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT
)

In [15]:
chain_language_example.run("""
What are the various products supplied?
""")



> Entering new  chain...
Generated Cypher:
MATCH (p:Product) RETURN p.productName as Product
Full Context:
[{'Product': 'Chai'}, {'Product': 'Chang'}, {'Product': 'Aniseed Syrup'}, {'Product': "Chef Anton's Cajun Seasoning"}, {'Product': "Chef Anton's Gumbo Mix"}, {'Product': "Grandma's Boysenberry Spread"}, {'Product': "Uncle Bob's Organic Dried Pears"}, {'Product': 'Northwoods Cranberry Sauce'}, {'Product': 'Mishi Kobe Niku'}, {'Product': 'Ikura'}]

> Finished chain.


"The various products supplied are Chai, Chang, Aniseed Syrup, Chef Anton's Cajun Seasoning, Chef Anton's Gumbo Mix, Grandma's Boysenberry Spread, Uncle Bob's Organic Dried Pears, Northwoods Cranberry Sauce, Mishi Kobe Niku, and Ikura."

In [ ]:
chain_language_example.run("Find the produce supplier for Northwoods Cranberry Sauce")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product {productName: "Northwoods Cranberry Sauce"})-[:PART_OF]->(c:Category {categoryName: "Produce"})-[:SUPPLIES]->(s:Supplier)
RETURN s.companyName
Full Context:
[]

> Finished chain.


"I'm sorry, I can't find the produce supplier for Northwoods Cranberry Sauce."

In [ ]:
chain_language_example.run("Find the produce suppliers")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Supplier)-[:SUPPLIES]->(p:Product)-[:PART_OF]->(c:Category {categoryName:"Produce"})
RETURN s.companyName as Company
Full Context:
[{'Company': "G'day"}, {'Company': 'Tokyo Traders'}, {'Company': 'Plutzer Lebensmittelgroßmärkte AG'}, {'Company': "Mayumi's"}, {'Company': "Grandma Kelly's Homestead"}]

> Finished chain.


"The produce suppliers are G'day, Tokyo Traders, Plutzer Lebensmittelgroßmärkte AG, Mayumi's, and Grandma Kelly's Homestead."

In [ ]:
chain_language_example.run("What categories of food does each supplier supply?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Supplier)-->(:Product)-->(c:Category)
RETURN s.companyName as Company, collect(distinct c.categoryName) as Categories
Full Context:
[{'Company': 'Exotic Liquids', 'Categories': ['Beverages', 'Condiments']}, {'Company': 'New Orleans Cajun Delights', 'Categories': ['Condiments']}, {'Company': "Grandma Kelly's Homestead", 'Categories': ['Produce', 'Condiments']}, {'Company': 'Tokyo Traders', 'Categories': ['Meat/Poultry', 'Produce', 'Seafood']}, {'Company': "Cooperativa de Quesos 'Las Cabras'", 'Categories': ['Dairy Products']}, {'Company': "Mayumi's", 'Categories': ['Produce', 'Seafood', 'Condiments']}, {'Company': 'Pavlova', 'Categories': ['Seafood', 'Beverages', 'Meat/Poultry', 'Condiments', 'Confections']}, {'Company': 'Specialty Biscuits', 'Categories': ['Confections']}, {'Company': 'PB Knäckebröd AB', 'Categories': ['Grains/Cereals']}, {'Company': 'Refrescos Americanas LTDA', 'Categories': ['Beverages']}, {'Comp

"The following is a list of the categories of food that each supplier supplies:\n\nExotic Liquids: Beverages, Condiments\nNew Orleans Cajun Delights: Condiments\nGrandma Kelly's Homestead: Produce, Condiments\nTokyo Traders: Meat/Poultry, Produce, Seafood\nCooperativa de Quesos 'Las Cabras': Dairy Products\nMayumi's: Produce, Seafood, Condiments\nPavlova: Seafood, Beverages, Meat/Poultry, Condiments, Confections\nSpecialty Biscuits: Confections\nPB Knäckebröd AB"

In [ ]:
chain_language_example.run("Who supplies Sasquatch Ale")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product {productName: "Sasquatch Ale"})-[:SUPPLIES]->(s:Supplier)
RETURN s.companyName
Full Context:
[]

> Finished chain.


"I'm not sure who supplies Sasquatch Ale."

In [ ]:
chain_language_example.run("What kind of node exists")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n) RETURN n.labels AS Labels
Full Context:
[{'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'Labels': None}, {'La

'There are no nodes.'